<a href="https://colab.research.google.com/github/ebahtiyar/ki_baglacinin_yazimi_deep_learningg/blob/main/ki_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

In [2]:
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation , Input,RepeatVector, TimeDistributed,Bidirectional
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,"/content/drive/My Drive/project")
import os
from sklearn.model_selection import train_test_split
import numpy as np
import random as r
import punc_preprocessing as punc
import pickle
import warnings
import gensim
from gensim.models import Word2Vec
import functions as f
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
from keras.models import load_model

Mounted at /content/drive


**Functions**

In [3]:
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

def find_de_da_ki():
    data = punc.load_data("/content/drive/My Drive/project/database.db","Total_senteces_F1")
    c_data = []
    for line in data:
        words = line.split()
        if len(words) < 35:
          c_data.append(line)
    da_de = []
    ki = []
    for i in c_data:
        i = i.replace("’","")
        i = re.sub(r'[^\w\s]', '', i)
        i = i.replace("madem ki","mademki")
        i = i.replace("meğer ki","meğerki")
        i = i.replace("oysa ki","oysaki")
        try:
            if i.find("da") != -1 :
                index  = i.find("da")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index + 2] == " " and i.find("iki") == -1 and i.find("eski") == -1:
                    ki.append(i)
                                      
        except:
            pass
    return da_de,ki

In [4]:
def data_simplify_conjunction():
    da_de,ki = find_de_da_ki()
    
    sp_da_de = []
    sp_ki = []
    
    adj_da_de = []
    adj_ki = []
    
    
    for i in da_de:
        try:
            if i.find("da") != -1:
                index = i.find("da")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)           
        except:
            pass
        
    for i in ki:
        try:
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index-1] == " ":
                    sp_ki.append(i)
                else:
                    adj_ki.append(i)         
        except:
            pass
        
    return sp_da_de,sp_ki,adj_da_de,adj_ki  

In [5]:
def word_tokenizer(line):
        line = re.sub(r'[^\w\s]', '', line)
        words = line.split()
        r_words = []
        try:
            for pos,word in enumerate(words):
                n = ""
                if pos > 0:
                   if word == "ki":
                      before_word = words[pos-1]
                      n = before_word + " ki"
                      r_words.remove(before_word)
                      r_words.append(n)
                   else:
                       r_words.append(word)
                else:
                    r_words.append(word)
        except:
            pass
                
        return r_words

In [6]:
def createTokenizer(data,Idx):
  output = []
  for line in data:
      wordIndices = []
      words = word_tokenizer(line)
      for word in words:
        if word in Idx:
          wordIdx = Idx[word].index
        elif word.lower() in Idx:
          wordIdx = Idx[word.lower()].index
        else:
          wordIdx = Idx['UNK'].index
        wordIndices.append(wordIdx)
      output.append(wordIndices)
  return output

In [7]:
def createTokenizer1(data,Idx):
  output = []
  for line in data:
      wordIndices = []
      words = word_tokenizer(line)
      for word in words:
        if word in Idx:
          wordIdx = Idx[word]
        elif word.lower() in Idx:
          wordIdx = Idx[word.lower()]
        else:
          wordIdx = Idx['UNK']
        wordIndices.append(wordIdx)
      output.append(wordIndices)
  return output

In [8]:
def generator_training_output(line):
    
        label = ""
        r_words = word_tokenizer(line)
                    
        for word in r_words:
            if word.find("ki") > 0:
                if len(word) > 1:
                    if word[len(word) - 2] == "k":
                        label = label + "E "
                    else:
                         label = label + "O "
                else:
                    label = label + "O "
            else:
                label = label + "O "
        label = label.strip()
        
          
        return label

In [9]:
def generator_training_input(data):
    training_input = []
    training_label = []
    for line in data:
        line = line.strip()
        training_input.append(line)
        r_words = word_tokenizer(line)
        label = ""
        for word in r_words:
            label = label + "O "
        training_label.append(label)
        
        b_da = [m.start() for m in re.finditer('ki ', line)]
        k  = 0
        for pos in b_da:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' k')
                    k = k + 1
                    
        training_input.append(line)
        label1 = generator_training_output(line)
        training_label.append(label1)
        
    return training_input,training_label

**Data Preliminary**

In [ ]:
sp_da_de,sp_ki,adj_da_de,adj_ki = data_simplify_conjunction()
data = adj_ki + sp_ki
ki_train , ki_test = train_test_split(data,test_size=0.1)
input_data = ki_train
input_data = f.unique(input_data)
training_input,training_label = generator_training_input(input_data)

s = list(zip(training_input,training_label))
r.shuffle(s)
training_input,training_label = zip(*s)


In [279]:
print("Total Separate Ki:" + str(len(sp_ki)))
print("Total Adjacent  Ki:" + str(len(adj_ki)))
print("Total Trainig Sentences: " + str(len(training_input)))
print("Total Testing Sentences: " + str(len(ki_test)))
print("İnput Sentence:",training_input[4321])
print("Label Sentence:",training_label[4321])

Total Separate Ki:62710
Total Adjacent  Ki:213981
Total Trainig Sentences: 416614
Total Testing Sentences: 27670
İnput Sentence: Toksöz ailesinin sahipliğinde ki Sağra çalışanlarının sendikası Genel İşten şikayetler aldık
Label Sentence: O O E O O O O O O O


**Pre Training using Word2Vec Model**

In [ ]:
traning_token =[]
for i in training_input:
  i = i.lower()
  traning_token.append(word_tokenizer(i))
traning_token.append(["UNK","PAD"])

In [ ]:
Word2Vec_model = Word2Vec(min_count=1)
Word2Vec_model.build_vocab(traning_token)  # prepare the model vocabulary
Word2Vec_model.train(traning_token, total_examples=Word2Vec_model.corpus_count, epochs= 100)  # train word vectors

(493322283, 531742000)

In [ ]:
Word2Vec_model.save("/content/drive/MyDrive/project/savedd_model/Word2Vec_model_ki")

In [ ]:
Word2Vec_model=Word2Vec.load("/content/drive/MyDrive/project/savedd_model/Word2Vec_model_ki")
pretrained_weights = Word2Vec_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


**Tokinezer and Padding Processing**

In [ ]:
x_train_tokenizer = createTokenizer(training_input,Word2Vec_model.wv.vocab)

labelSet = set()
for line in training_label:
  words = word_tokenizer(line)
  for word in words:
    labelSet.add(word.lower())

sorted_labels = sorted(list(labelSet), key=len)
# Create mapping for labels
label2Idx = {}
if len(label2Idx) == 0:
  label2Idx["PAD"] = len(label2Idx)
  label2Idx["UNK"] = len(label2Idx)
for label in sorted_labels:
  label2Idx[label] = len(label2Idx)
idx2Label = {v: k for k, v in label2Idx.items()}

y_train_tokenizer = createTokenizer1(training_label,label2Idx)

In [ ]:
max_in_len = 34
max_out_len = 34
x_train_pad = pad_sequences(x_train_tokenizer, max_in_len, padding = "post")
y_train_pad = pad_sequences(y_train_tokenizer, max_out_len, padding = "post")

x_train_pad = x_train_pad.reshape(*x_train_pad.shape, 1)
y_train_pad = y_train_pad.reshape(*y_train_pad.shape, 1)
print("Max Length of Train Sentences:" + str(max_in_len) + " Max Length of Train Labels:" + str(max_out_len))

**Bidirectional Model**

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
callbacks = [checkpoint]

In [ ]:
model_main = Sequential()
model_main.add(Input(shape=(max_in_len,)))
model_main.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size,weights=[pretrained_weights]))
model_main.add(Bidirectional(LSTM(units=emdedding_size,return_sequences=False)))
model_main.add(RepeatVector(max_out_len))
model_main.add(Bidirectional(LSTM(emdedding_size, return_sequences=True, dropout=0.1)))
model_main.add(TimeDistributed(Dense(len(label2Idx)+1)))
model_main.add(Activation('softmax'))
model_main.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])

In [ ]:
model_main.fit(x_train_pad, y_train_pad, validation_split = 0.2, epochs= 10 , callbacks=callbacks, batch_size = 256)

In [ ]:
model_main.save('/content/drive/My Drive/project/savedd_model/model_ki_w2v_1.h5')

**Testting**

In [16]:
model = load_model('/content/drive/My Drive/project/savedd_model/model_ki_w2v_1.h5')

In [17]:
idx2Label = {0 :'PAD', 1:'UNK' , 2: 'o' , 3: 'e'}
Word2Vec_model=Word2Vec.load("/content/drive/MyDrive/project/savedd_model/Word2Vec_model_ki")

In [12]:
def prediction(model_main,sample,Word2Vec_model,idx2Label):
  sample_pad = []
  words = word_tokenizer(sample.lower())
  for word in words:
      if word in Word2Vec_model.wv.vocab:
          
         sample_pad.append(Word2Vec_model.wv.vocab[word].index)
      else:
         sample_pad.append(Word2Vec_model.wv.vocab["UNK"].index)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')

  pred = model_main.predict(sample_pad)
  label = [idx2Label[np.argmax(x)] for x in pred[0]]
  output_label = ""
  for i in label:
      output_label = output_label + i + " "

  pred_sent = ""
  for i in zip(words,label):
    if i[1] == "o":
      pred_sent = pred_sent +  i[0] + " "
    elif i[1] == "e":
        if i[0].find(" ki") > 0:
            new_word = replace_str_index(i[0],index=i[0].find(" ki"),replacement='')
            pred_sent = pred_sent + new_word + " "
        elif i[0].find(" ") == -1 and i[0].find("ki") == len(i[0]) - 2:
            new_word = replace_str_index(i[0],i[0].find("ki"),replacement=' k')
            pred_sent = pred_sent + new_word + " "
        else:
            pred_sent = pred_sent + i[0] + " "
            
    elif i[0][1] == "PAD":
        continue
  pred_sent = pred_sent.strip()

  r_words = word_tokenizer(pred_sent)
  return pred_sent

In [11]:
def generator_testing_input(data):
    training_input = []
    training_label = []
    for line in data:
        line = line.strip()
        b_da = [m.start() for m in re.finditer('ki ', line)]
        k  = 0
        for pos in b_da:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' k')
                    k = k + 1
                    
        training_input.append(line)
    
    return training_input

In [288]:
sp_ki_test = punc.n_punc(ki_test," ki ")

test = r.choices(sp_ki_test,k=1000) + r.choices(adj_ki,k = 1000)
testing_input = generator_testing_input(test)

In [289]:
pred = []
pred_last = []
wrong = []
wrong_pos = []
for line in testing_input:
    pred.append(prediction(model,line.lower(),Word2Vec_model,idx2Label).strip())
   
k = 0
for pos,line in enumerate(test):
    line = line.lower()
    line = line.replace("i̇","i")
    if line.lower() == pred[pos]:
        k = k + 1
    else:
      wrong.append(pred[pos])
      wrong_pos.append(pos)

In [290]:
rand = r.randrange(100)
print("Score: " + str(k) +" in " + str(len(testing_input)) + " sentences")
print("Orginal: "+test[rand].lower())
print("Wrong: "+testing_input[rand].lower())
print("Prediction: "+pred[rand])

Score: 1855 in 2000 sentences
Orginal: herkesin kendi içerisinde büyük planları var tabii ki isterim
Wrong: herkesin kendi içerisinde büyük planları var tabiiki isterim
Prediction: herkesin kendi içerisinde büyük planları var tabii ki isterim


**CALCULATE SCORE**

In [286]:
def prediction_score(model_main,sample,Word2Vec_model,idx2Label):
  sample_pad = []
  words = word_tokenizer(sample.lower())
  for word in words:
      if word in Word2Vec_model.wv.vocab:
          
         sample_pad.append(Word2Vec_model.wv.vocab[word].index)
      else:
         sample_pad.append(Word2Vec_model.wv.vocab["UNK"].index)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')

  pred = model_main.predict(sample_pad)
  label = [idx2Label[np.argmax(x)] for x in pred[0]]

  return label

In [291]:
pred_label = []

for line in testing_input:
    pred_label.append(prediction_score(model,line.lower(),Word2Vec_model,idx2Label))

In [297]:
pred_label_l = []

for pred in pred_label:
    t = []
    for i in pred:
        if i != "PAD":
           if i == "e":
              i = "DEC"
           elif i == "o":
              i = "CON"
           t.append(i.upper())
        else:
          break
    pred_label_l.append(t)

In [298]:
test_label = []

for line in testing_input:
    test_label.append(generator_training_output(line).lower())
test = []
for line in test_label:
     line = line.upper()
     x =  []
     for l in line.split():
         if l == "O":
            i = "CON"
         else:
            i = "DEC"
         x.append(i)
     test.append(x)

test_x = []
text_label = []
for i in range(len(test)):
  if len(test[i]) == len(pred_label_l[i]):
      test_x.append(test[i])
      text_label.append(pred_label_l[i]) 

In [299]:
score = classification_report(test_x,text_label)
print(score)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DEC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

          EC       0.98      0.93      0.96      2168
          ON       0.95      0.93      0.94      3588

   micro avg       0.96      0.93      0.95      5756
   macro avg       0.97      0.93      0.95      5756
weighted avg       0.97      0.93      0.95      5756

